In [3]:

import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


import pickle

In [4]:
with open('hwdata.pkl', 'rb') as f:
    hwdata = pickle.load(f)
    print(hwdata.keys())

dict_keys(['Vocabulary', 'XTrain', 'yTrain', 'XTest', 'yTest'])


In [5]:
# print(hwdata)
vocabulary = hwdata["Vocabulary"]
XTrain = hwdata["XTrain"]
yTrain = hwdata["yTrain"]
XTest = hwdata["XTest"]
yTest = hwdata["yTest"]

type(hwdata["yTrain"])
hwdata["XTrain"].shape

(450, 26050)

In [6]:
"""
    Compute the probability of P(X|Y).

    :param
        XTrain: numpy array of size [num_samples, feat_dim]
          where num_samples is the number of samples
          and feat_dim is the dimension of features
        yTrain: numpy array of size [num_samples, 1]
        a: default to 0.001
        b: default to 0.9

    :return: 
        D: numpy array of size [2, vocab_size] where
          vocab_size is the size of vocabulary
"""
def NB_XGivenY(XTrain, yTrain, a=0.001, b=0.9): 
    num_samples, feat_dim = XTrain.shape
    num_classes = 2                      # since we have just 2 classes: economist and onion
    economist = yTrain[:,0] == 1         # boolean masks which are set to true if the class = 1 i.e. the i-th document belongs to the economist
    onion = yTrain[:,0] == 2             # boolean masks which are set to true if the class = 2 i.e. the i-th document belongs to the onion

    # boolean indexing of arrays so that we can pick subsets of the data
    economist_matrix = XTrain[economist]    # subset of the training documents which belong to the economist
    onion_matrix = XTrain[onion]            # subset of the training documents which belong to the onion

    # for each word in the vocabulary, we sum the number of times it has appeared in economist articles. We add a +1 to avoid 0 probabilty
    # we divide by the total number of articles in the economist to get the probability of each word in the economist
    # we do the same for onion
    # we divide the economist sum by the total number of articles in the economist (again adding a +1 so that we don't divide 
    # by 0 in case it happens there are no economist articles) to get the probability that each word occurs in an economist article
    D = np.array([
                    (np.sum(economist_matrix, axis=0) + a) / (economist_matrix.shape[0] + b),
                    (np.sum(onion_matrix, axis=0) + a)     / (onion_matrix.shape[0] + b),        
                ])
    
    return D
    # raise NotImplementedError 
    # return D


In [7]:
# estprob = NB_XGivenY(hwdata["XTrain"], hwdata["yTrain"])
# print(estprob)
economist = hwdata["yTrain"][:,0] == 1
onion = hwdata["yTrain"][:,0] == 2
vocabulary = hwdata["Vocabulary"]
# print(economist)
economist_matrix = hwdata["XTrain"][economist]
print(economist_matrix.shape[0])

onion_matrix = hwdata["XTrain"][onion]
print(onion_matrix.shape)


350
(100, 26050)


In [8]:
print(onion_matrix[0])
# print(vocabulary[int(min(np.sum(economist_matrix, axis = 0)))])
print(max( (np.sum(economist_matrix, axis = 0) + 1) / (1 + economist_matrix.shape[0]) ) )

[1. 1. 0. ... 0. 1. 0.]
0.9544159544159544


In [9]:
D = NB_XGivenY(hwdata["XTrain"], hwdata["yTrain"])
#largest value of the first row of D
print(np.max(D[0,:]))
print(vocabulary[np.argmax(D[0,:])])
print(np.max(D[1,:]))
print(vocabulary[np.argmax(D[1,:])])
print(sum(onion_matrix[:,1]))
print(sum(economist_matrix[:,31])/350.0)


0.9518409803362782
['the']
0.9910901883052528
['a']
100.0
0.9542857142857143


In [10]:
D

array([[5.69965802e-01, 9.48991166e-01, 2.99233400e-01, ...,
        2.84981476e-06, 3.04933029e-01, 2.84981476e-06],
       [6.73944500e-01, 9.91090188e-01, 4.36085233e-01, ...,
        9.92071358e-03, 3.17155600e-01, 9.91080278e-06]])

In [11]:
"""
    Compute the probability of P(Y).

    :param
        yTrain: numpy array of size [num_samples, 1]

    :return: 
        p: a scalar for the probability of P(Y = 1)
"""
def NB_YPrior(yTrain):
    economist = yTrain[:,0] == 1                        # class is economist
    num_samples = yTrain.shape[0]                       
    num_economist_articles = yTrain[economist].shape[0] # number of economist articles
    p = num_economist_articles / num_samples            # probability of P(Y = 1)
    return p

    # raise NotImplementedError
    # return p

In [12]:
p = NB_YPrior(hwdata["yTrain"])
p

0.7777777777777778

In [13]:
def NB_Classify(D, p, X):
    num_samples = X.shape[0]
    y = np.zeros((num_samples))
    log_D = np.log(D) 
    prob = np.dot(X, log_D.T) + np.log(np.array([p, 1 - p])) # we work in the logspace for numerical precision by taking log of small values like probabilities

    for i in range(num_samples):
        max_index = np.argmax(prob[i, :]) # choose economist or onion based on whose probability is higher
        y[i] = max_index + 1 # adding 1 cause 1 and 2 are the classes

    # raise NotImplementedError
    return y.reshape(-1, 1)



In [14]:
Xtest = hwdata["XTest"]
yTest = hwdata["yTest"]
Xtest.shape[0]

153

In [15]:
yPred = NB_Classify(D, p, Xtest)
yPred.shape

(153, 1)

In [16]:
"""
Compute the accuracy of predictions.

:param
    yHat: numpy array of size [num_samples, 1]
    yTruth: numpy array of size [num_samples, 1]

:return:
    acc: a scalar for the accuracy
"""
def NB_ClassificationAccuracy(yHat, yTruth):
    acc = np.sum(yHat == yTruth) / yTruth.shape[0]
    return acc
    
    # raise NotImplementedError
    # return acc

### 2. Reporting results on test data

In [17]:
acc = NB_ClassificationAccuracy(yPred, yTest)
print(acc)

0.9738562091503268
